<a href="https://colab.research.google.com/github/ykim71/georgia_analysis/blob/main/(2)_Georgia_place_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# need access to the folder(s) below:

#%cd /content/drive/MyDrive/Georgia/


# Location analysis

## Prep for place analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
df_temp = df.sample(10)

In [ ]:
# lemmatization if needed
import spacy
from spacy import displacy

sp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])


In [ ]:

def text_ner_place(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if entity.label_ == "FAC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "GPE":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "LOC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return set(ner_list)


In [ ]:

def text_ner_place2(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if (entity.label_ == "FAC") or  (entity.label_ == "GPE") or (entity.label_ == "LOC") or (entity.label_ == "ORG") :
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return set(ner_list)


In [ ]:

def text_ner_person(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if (entity.label_ == "PERSON") :
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return set(ner_list)


In [ ]:

def text_ner_all(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
      ner_list.append(entity.text + " (" + entity.label_ + ")")
  return set(ner_list)


In [ ]:
df['ner_places'] = df['article_final'].astype(str).apply(text_ner_place)

In [ ]:
df['ner_places_orgs'] = df['article_final'].astype(str).apply(text_ner_place2)

In [ ]:
df['ner_person'] = df['article_final'].astype(str).apply(text_ner_person)

In [ ]:
df.to_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner3_lem.pkl') # commented this line to prevent from saving it

## Find rough location from spaCy

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
df_ner = df.loc[df['ner_places'] != 'set()']

In [ ]:
print(len(df))
print(len(df_ner))

In [ ]:
df_expand = df_ner['ner_places'].str.split(',').explode().str.strip('{} ')


In [ ]:
df_expand = pd.DataFrame(df_expand)


In [ ]:
df_expand['ner_places_clean'] = df_expand['ner_places'].str.replace(r"\'","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r" \"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\(.*\)","")

In [ ]:
df_expand_count = df_expand.groupby('ner_places_clean').count().reset_index()

In [ ]:
df_freq_n5 = df_expand_count[df_expand_count["ner_places"] >= 5]
df_freq_n5.to_excel('df_freq_n5_ner_v2.xlsx')

In [ ]:
# # compare place entities with v1

# df_freq_n5_v2 = pd.read_excel('df_freq_n5_ner_v2.xlsx')
# df_freq_n5_v1 = pd.read_excel('df_freq_n5_ner.xlsx')

# df_freq_n5_v1['version'] = 'v1'
# df_freq_n5_v2['version'] = 'v2'

# df_freq_n5_merge = pd.merge( df_freq_n5_v1,df_freq_n5_v2, how="outer", on=["ner_places_clean"])

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis/Validation_test

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner3_lem.pkl')


In [ ]:
df_ner_place_org = df.loc[df['ner_places_orgs'] != 'set()']
df_ner_person = df.loc[df['ner_person'] != 'set()']

In [ ]:
df_ner_place_org_expand = df_ner_place_org['ner_places_orgs'].explode()
df_ner_person_expand = df_ner_person['ner_person'].explode()


In [ ]:
df_ner_place_org_expand = pd.DataFrame(df_ner_place_org_expand)
df_ner_person_expand = pd.DataFrame(df_ner_person_expand)


In [ ]:
df_ner_place_org_expand['ner_clean'] = df_ner_place_org_expand['ner_places_orgs'].str.replace(r" \(.*\)","")
df_ner_person_expand['ner_clean'] = df_ner_person_expand['ner_person'].str.replace(r" \(.*\)","")

In [ ]:
df_ner_place_org_expand = df_ner_place_org_expand.dropna()
df_ner_person_expand = df_ner_person_expand.dropna()

In [ ]:
df_ner_place_org_expand['ner_clean'] = df_ner_place_org_expand['ner_clean'].str.strip()
df_ner_person_expand['ner_clean'] = df_ner_person_expand['ner_clean'].str.strip()

In [ ]:
df_ner_person_expand

In [ ]:
df_ner_place_org_count = df_ner_place_org_expand.groupby('ner_clean').count().reset_index()
df_ner_person_count = df_ner_person_expand.groupby('ner_clean').count().reset_index()

In [ ]:
print(len(df_ner_place_org_count))
print(len(df_ner_person_count))

In [ ]:
df_freq_place_org_n5 = df_ner_place_org_count[df_ner_place_org_count["ner_places_orgs"] >= 5]
df_freq_person_n5 = df_ner_person_count[df_ner_person_count["ner_person"] >= 5]


In [ ]:
print(len(df_freq_place_org_n5))
print(len(df_freq_person_n5))

In [ ]:
df_freq_place_org_n5.to_excel('df_freq_place_org_n5.xlsx')

In [ ]:
df_freq_person_n5.to_excel('df_freq_person_n5.xlsx')

## Find sentence example of location

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis

import pandas as pd

#ner_places = pd.read_csv('Georgia_ner_places.csv') # previous data
#ner_places = pd.read_csv('Georgia_ner_places_v2.csv') # updated data
ner_places = pd.read_excel('Georgia_place_review_final_validation.xlsx') # updated data

In [ ]:
#ner_places['ner_places_clean'] = ner_places['ner_places_clean'].str.rstrip()

In [ ]:
ner_places_raw = list(set(ner_places['ner_places_clean2'].tolist()))

In [ ]:
import pandas as pd
import re

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')

# remove bracket and quotes
df['ner_places_clean'] = df['ner_places'].apply(lambda x: re.sub(r'[{}]', '', x))
#df['ner_places_clean'] = df['ner_places_clean'].str.replace(r" \(GPE\)","")
df['ner_places_clean'] = df['ner_places_clean'].str.replace(r"\', \'", " ")
df['ner_places_clean'] = df['ner_places_clean'].str.rstrip("'")
df['ner_places_clean'] = df['ner_places_clean'].str.lstrip("'")



In [ ]:
import nltk
nltk.download('punkt')

In [ ]:

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]

word_bigrams_done = list(set(bigrams(ner_places_raw)))
print(word_bigrams_done)
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(ner_places_raw)
print(word_unigrams_done)
print(len(word_unigrams_done))


In [ ]:
# only for what spaCy found

df['place_ner1'] = df['ner_places_clean'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['place_ner2'] = df['ner_places_clean'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:

# ner identification

df_uni2 = df[(df['place_ner1'].str.len()) >= 1]
df_bi2 = df[(df['place_ner2'].str.len()) >= 1]

df_uni2 = df_uni2[['id', 'source', 'place_ner1', 'article_final']]
df_bi2 = df_bi2[['id', 'source', 'place_ner2', 'article_final']]

df_uni_df2 = df_uni2.explode('place_ner1')
df_bi_df2 = df_bi2.explode('place_ner2')

df_uni_df2 = df_uni_df2.rename(columns={'place_ner1': 'place_ner'})
df_bi_df2 = df_bi_df2.rename(columns={'place_ner2': 'place_ner'})

df_ng_all2 = pd.concat([df_uni_df2, df_bi_df2])

In [ ]:
df_ng_all2

In [ ]:
df_sentence = df_ng_all2[['id', 'article_final']].drop_duplicates(subset=['id', 'article_final'])

In [ ]:
# split an article into sentences

from nltk.tokenize import sent_tokenize
by_sentence = df_sentence['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


In [ ]:
merge_uni = pd.merge(df_ng_all2, by_sentence_df, left_index=True, right_index=True)


In [ ]:
merge_uni['search_words'] = merge_uni.apply(lambda x: x.place_ner in x.sentence_each, axis=1)

merge_uni_TRUE = merge_uni[merge_uni['search_words']==True]

In [ ]:
merge_uni_TRUE_st = merge_uni_TRUE[['id','source','place_ner','sentence_each']]

In [ ]:
merge_temp = merge_uni_TRUE_st.groupby('place_ner').count() #

In [ ]:
merge_temp

In [ ]:
# articles mentioned a location less than 3
merge_temp_less10 = merge_temp[merge_temp['sentence_each'] < 10].reset_index()


In [ ]:
merge_temp_less10_places = merge_temp_less10.place_ner.values.tolist()


In [ ]:
len(merge_temp_less10_places)

In [ ]:
merge_uni_TRUE_st_less10 = merge_uni_TRUE_st[merge_uni_TRUE_st['place_ner'].isin(merge_temp_less10_places)]


In [ ]:
# articles mentioned a location more than 3 --> random sample

merge_uni_TRUE_st_rest = merge_uni_TRUE_st[~merge_uni_TRUE_st['place_ner'].isin(merge_temp_less10_places)]
merge_uni_TRUE_st_more10 = merge_uni_TRUE_st_rest.groupby('place_ner').apply(lambda x: x.sample(10, replace=False)).reset_index(drop=True)


In [ ]:
merge_uni_TRUE_st_more10

In [ ]:
# merge 3 samples + those with less than 3 
merge_all = pd.concat([merge_uni_TRUE_st_more10, merge_uni_TRUE_st_less10]).reset_index()

In [ ]:
merge_all = merge_all.sort_values('place_ner')

In [ ]:
merge_all['sentenceID'] = merge_all.groupby('place_ner')['place_ner'].rank(method='first')


In [ ]:
merge_all

In [ ]:
merge_all_pivot = merge_all.pivot(index='place_ner', columns = 'sentenceID', values='sentence_each').reset_index()


In [ ]:
merge_all_pivot.rename(columns={1:'s1', 2:'s2', 3:'s3', 4:'s4', 5:'s5', 6:'s6', 7:'s7', 8:'s8', 9:'s9', 10:'s10'}, inplace=True)


In [ ]:
len(ner_places_raw)

In [ ]:
len(list(set(ner_places_raw) - set(merge_all_pivot['place_ner'].values.tolist())))

In [ ]:
#list(set(ner_places_raw) - set(merge_all_pivot['place_ner'].values.tolist()))

In [ ]:
# merge_temp_final['sent_1'] = merge_temp_final['sent_1'].str.replace('\n', ' ')
# merge_temp_final['sent_2'] = merge_temp_final['sent_2'].str.replace('\n', ' ')
# merge_temp_final['sent_3'] = merge_temp_final['sent_3'].str.replace('\n', ' ')

In [ ]:
ner_places['place_ner'] = ner_places['ner_places_clean2']
ner_places_st = ner_places[['place_ner', 'address', 'address_ga', 'address_usa']]

In [ ]:
merge_all_pivot_merge = pd.merge(merge_all_pivot, ner_places_st, on="place_ner", how='left')


In [ ]:
# Mt. Pisgah
merge_all_pivot_merge.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/COPY_Georgia_ner_places_validation_10_sent_example.xlsx')

In [ ]:
import pandas as pd

merge_all_pivot_merge = pd.read_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Georgia_ner_places_validation_10_sent_example.xlsx')


In [ ]:
len(merge_all_pivot_merge)

In [ ]:
len(merge_all_pivot_merge)-2686

In [ ]:
# those only with more than 10 examples
merge_all_pivot_merge_filter = merge_all_pivot_merge.dropna(subset=['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10' ])

In [ ]:
merge_all_pivot_merge_filter.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Georgia_ner_places_validation_10_sent_example.xlsx')

In [ ]:
## find examples of PERSON ner

%cd /content/drive/MyDrive/Georgia/Locations Analysis

import pandas as pd
import re

ner_person = pd.read_excel('df_person_results_GA_TRUE_wide.xlsx')
df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner3_lem.pkl')


In [ ]:
df

In [ ]:
ner_person_raw = list(set(ner_person['ner_original'].tolist()))

In [ ]:

import nltk
nltk.download('punkt')

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]

word_bigrams_done = list(set(bigrams(ner_person_raw)))
print(word_bigrams_done)
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(ner_person_raw)
print(word_unigrams_done)
print(len(word_unigrams_done))



In [ ]:
# remove bracket and quotes
df['ner_places_orgs_clean'] = df['ner_places_orgs'].astype(str).apply(lambda x: re.sub(r'[{}]', '', x))
#df['ner_places_clean'] = df['ner_places_clean'].str.replace(r" \(GPE\)","")
df['ner_places_orgs_clean'] = df['ner_places_orgs_clean'].str.replace(r"\', \'", " ")
df['ner_places_orgs_clean'] = df['ner_places_orgs_clean'].str.rstrip("'")
df['ner_places_orgs_clean'] = df['ner_places_orgs_clean'].str.lstrip("'")



In [ ]:
# remove bracket and quotes
df['ner_person_clean'] = df['ner_person'].astype(str).apply(lambda x: re.sub(r'[{}]', '', x))
#df['ner_places_clean'] = df['ner_places_clean'].str.replace(r" \(GPE\)","")
df['ner_person_clean'] = df['ner_person_clean'].str.replace(r"\', \'", " ")
df['ner_person_clean'] = df['ner_person_clean'].str.rstrip("'")
df['ner_person_clean'] = df['ner_person_clean'].str.lstrip("'")



In [ ]:

# only for what spaCy found

df['person_ner1'] = df['ner_person_clean'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['person_ner2'] = df['ner_person_clean'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])



In [ ]:

# only for what spaCy found

df['person_ner3'] = df['ner_places_orgs_clean'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['person_ner4'] = df['ner_places_orgs_clean'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])



In [ ]:
# ner identification

df_temp1 = df[(df['person_ner1'].str.len()) >= 1]
df_temp2 = df[(df['person_ner2'].str.len()) >= 1]
df_temp3 = df[(df['person_ner3'].str.len()) >= 1]
df_temp4 = df[(df['person_ner4'].str.len()) >= 1]

df_temp1 = df_temp1[['id', 'source', 'person_ner1', 'article_final']]
df_temp2 = df_temp2[['id', 'source', 'person_ner2', 'article_final']]
df_temp3 = df_temp3[['id', 'source', 'person_ner3', 'article_final']]
df_temp4 = df_temp4[['id', 'source', 'person_ner4', 'article_final']]

df_temp1_exp = df_temp1.explode('person_ner1')
df_temp2_exp = df_temp2.explode('person_ner2')
df_temp3_exp = df_temp3.explode('person_ner3')
df_temp4_exp = df_temp4.explode('person_ner4')

df_temp1_exp = df_temp1_exp.rename(columns={'person_ner1': 'person_ner'})
df_temp2_exp = df_temp2_exp.rename(columns={'person_ner2': 'person_ner'})
df_temp3_exp = df_temp3_exp.rename(columns={'person_ner3': 'person_ner'})
df_temp4_exp = df_temp4_exp.rename(columns={'person_ner4': 'person_ner'})

df_ng_all2 = pd.concat([df_temp1_exp, df_temp2_exp, df_temp3_exp, df_temp4_exp])

df_sentence = df_ng_all2[['id', 'article_final']].drop_duplicates(subset=['id', 'article_final'])


In [ ]:
len(df_sentence)

In [ ]:

# split an article into sentences

from nltk.tokenize import sent_tokenize
by_sentence = df_sentence['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)

merge_uni = pd.merge(df_ng_all2, by_sentence_df, left_index=True, right_index=True)

merge_uni['search_words'] = merge_uni.apply(lambda x: x.person_ner in x.sentence_each, axis=1)

merge_uni_TRUE = merge_uni[merge_uni['search_words']==True]

In [ ]:
merge_uni_TRUE_st = merge_uni_TRUE[['id','source','person_ner','sentence_each']]
merge_temp = merge_uni_TRUE_st.groupby('person_ner').count() #


In [ ]:

# articles mentioned a location less than 10
merge_temp_less10 = merge_temp[merge_temp['sentence_each'] < 10].reset_index()
merge_temp_less10_places = merge_temp_less10.person_ner.values.tolist()


In [ ]:
merge_uni_TRUE_st_less10 = merge_uni_TRUE_st[merge_uni_TRUE_st['person_ner'].isin(merge_temp_less10_places)]


In [ ]:
len(merge_temp_less10_places)

In [ ]:

# articles mentioned a location more than 10 --> random sample

merge_uni_TRUE_st_rest = merge_uni_TRUE_st[~merge_uni_TRUE_st['person_ner'].isin(merge_temp_less10_places)]
merge_uni_TRUE_st_more10 = merge_uni_TRUE_st_rest.groupby('person_ner').apply(lambda x: x.sample(10, replace=False)).reset_index(drop=True)

In [ ]:
# merge 10 samples + those with less than 10 
merge_all = pd.concat([merge_uni_TRUE_st_more10, merge_uni_TRUE_st_less10]).reset_index()

In [ ]:

merge_all = merge_all.sort_values('person_ner')
merge_all['sentenceID'] = merge_all.groupby('person_ner')['person_ner'].rank(method='first')

merge_all_pivot = merge_all.pivot(index='person_ner', columns = 'sentenceID', values='sentence_each').reset_index()

merge_all_pivot.rename(columns={1:'s1', 2:'s2', 3:'s3', 4:'s4', 5:'s5', 6:'s6', 7:'s7', 8:'s8', 9:'s9', 10:'s10'}, inplace=True)


In [ ]:
merge_all_pivot

In [ ]:
ner_person

In [ ]:

ner_person['person_ner'] = ner_person['ner_original']

merge_all_pivot_merge = pd.merge(merge_all_pivot, ner_person, on="person_ner", how='left')


In [ ]:
merge_all_pivot_merge.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/Georgia_ner_person_validation_10_sent_example.xlsx')


In [ ]:
# those only with more than 10 examples
merge_all_pivot_merge_filter = merge_all_pivot_merge.dropna(subset=['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10' ])

merge_all_pivot_merge_filter.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/Georgia_ner_person_validation_10_sent_example_m10.xlsx')


In [ ]:
# exclude those that are in the ner place worksheet

ner_place = pd.read_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Georgia_ner_places_validation_10_sent_example.xlsx')

In [ ]:
ner_place_raw = list(set(ner_place['place_ner'].tolist()))
merge_all_pivot_merge_rest = merge_all_pivot_merge[~merge_all_pivot_merge['person_ner'].isin(ner_place_raw)]


In [ ]:
len(merge_all_pivot_merge)

In [ ]:
len(merge_all_pivot_merge_rest)

In [ ]:
merge_all_pivot_merge_rest2 = merge_all_pivot_merge_filter[~merge_all_pivot_merge_filter['person_ner'].isin(ner_place_raw)]


In [ ]:
len(merge_all_pivot_merge_rest2)

In [ ]:
merge_all_pivot_merge_rest2.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/Georgia_ner_person_validation_10_sent_example_m10_rest.xlsx')


## Look up location (w updated data) on articles

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis

In [ ]:
# data created by manual coding

import pandas as pd

location_final = pd.read_excel('Georgia_place_validation_counties.xlsx')

In [ ]:
location_final = location_final[location_final['final_selection']==1]

In [ ]:
location_final_st = location_final[['ner_place_clean','county_level', 'final_selection']]

In [ ]:
location_final_st = location_final_st.dropna()

In [ ]:
len(location_final_st)

In [ ]:
location_final_st

In [ ]:
# # multiple counties deleted
# location_final_st = location_final_st[location_final_st['county_level'].str.contains('Counties')==False]

In [ ]:
len(set(location_final_st.county_level.values.tolist()))

In [ ]:
location_final_list = list(set(location_final_st['ner_place_clean'].tolist()))

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
# remove bracket and quotes
df['ner_places_clean'] = df['ner_places'].apply(lambda x: re.sub(r'[{}]', '', x))
#df['ner_places_clean'] = df['ner_places_clean'].str.replace(r" \(GPE\)","")
df['ner_places_clean'] = df['ner_places_clean'].str.replace(r"\', \'", " ")
df['ner_places_clean'] = df['ner_places_clean'].str.rstrip("'")
df['ner_places_clean'] = df['ner_places_clean'].str.lstrip("'")



In [ ]:

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]

word_bigrams_done = list(set(bigrams(location_final_list)))
print(word_bigrams_done)
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(location_final_list)
print(word_unigrams_done)
print(len(word_unigrams_done))


In [ ]:
import numpy as np
df['place_ng1'] = df['article_final'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['place_ng2'] = df['article_final'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:

# only for what spaCy found

df['place_ner1'] = df['ner_places_clean'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['place_ner2'] = df['ner_places_clean'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:
# article identification

df_uni = df[(df['place_ng1'].str.len()) >= 1]
df_bi = df[(df['place_ng2'].str.len()) >= 1]

df_uni = df_uni[['id', 'source', 'place_ng1']]
df_bi = df_bi[['id', 'source', 'place_ng2']]

df_uni_df = df_uni.explode('place_ng1')
df_bi_df = df_bi.explode('place_ng2')

df_uni_df = df_uni_df.rename(columns={'place_ng1': 'place_ner'})
df_bi_df = df_bi_df.rename(columns={'place_ng2': 'place_ner'})


In [ ]:
df_ng_all = pd.concat([df_uni_df, df_bi_df])

In [ ]:
# ner identification

df_uni2 = df[(df['place_ner1'].str.len()) >= 1]
df_bi2 = df[(df['place_ner2'].str.len()) >= 1]

df_uni2 = df_uni2[['id', 'source', 'place_ner1']]
df_bi2 = df_bi2[['id', 'source', 'place_ner2']]

df_uni_df2 = df_uni2.explode('place_ner1')
df_bi_df2 = df_bi2.explode('place_ner2')

df_uni_df2 = df_uni_df2.rename(columns={'place_ner1': 'place_ner'})
df_bi_df2 = df_bi_df2.rename(columns={'place_ner2': 'place_ner'})


In [ ]:
df_ng_all2 = pd.concat([df_uni_df2, df_bi_df2])

In [ ]:
count = df_ng_all.groupby(['source']).count()

In [ ]:
count_20 = count[count['place_ner'] >= 20].reset_index()

In [ ]:
count2 = df_ng_all2.groupby(['source']).count()

In [ ]:
count2_20 = count2[count2['place_ner'] >= 20].reset_index()

In [ ]:
count_20

In [ ]:
count2_20

In [ ]:
count_20_id = list(set(count_20['source'].tolist()))

df_ng_all_m20 = df_ng_all[df_ng_all['source'].isin(count_20_id)]


In [ ]:
len(count_20_id)

In [ ]:
location_final_st = location_final_st.rename(columns={'ner_place_clean': 'place_ner'})


In [ ]:
df_ng_all_m20_county = pd.merge(df_ng_all_m20, location_final_st, how='left', on='place_ner')

In [ ]:
df_ng_all_m20_county_unique = df_ng_all_m20_county.drop_duplicates(subset=['id', 'source', 'county_level'])

In [ ]:
df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique.groupby(['source','county_level'])['id'].count().reset_index()

In [ ]:
df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique_count.rename(columns={'id': 'article_id_count'})


In [ ]:
df_ng_all_m20_county_unique_count.to_excel('Georgia_m20_county_unique_count_v2.xlsx')

## Take random samples for manual validation

In [ ]:
df_sample_20 = df.sample(20)

In [ ]:
df_sample_20.to_excel('Georgia_20_articles_for_place_review.xlsx')

In [ ]:
df_sample_15 = df.sample(15)

In [ ]:
df_sample_15.to_excel('Georgia_15_articles_for_place_review.xlsx')

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis/Validation_test

In [ ]:
import pandas as pd

rnd1 = pd.read_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/Copy of Georgia_15_articles_for_place_review.xlsx')
rnd2 = pd.read_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/Copy of Georgia_20_articles_for_place_review.xlsx')

In [ ]:
rnd1_st = rnd1[['id', 'ner_places', 'article_final']]
rnd2_st = rnd2[['id', 'ner_places', 'article_final']]

In [ ]:
set_merge = pd.concat([rnd1_st, rnd2_st])

In [ ]:
temp = 'MARIETTA — The renovation of northeast Cobb’s Gritters Library, originally budgeted as a $2.9 million project, could soon cost $10.5 million after Tuesday’s Board of Commissioners meeting. The ballooning budget is a result not only of rising construction costs, but of a project that’s now envisioned as a full rebuild of the aging library. The board was expected to vote Tuesday on approving a $2.5 million transfer to fill out the budget, though support for the move appeared to waver at a Monday morning work session. Tucked into the woods alongside Shaw Park off of Canton Road, Gritters, according to Library Director Helen Poyer, “sits in a swampland” and is “an old cedar building that’s in very poor condition.” Major features of the rebuild will include building out reading and study rooms, expanding an existing multipurpose room, creating an outdoor programming space, and consolidating services into a one-stop service desk. Poyer added that in 2014, when the library system lobbied for the building to be put on the 2016 SPLOST list, she already knew a complete rebuild would be required and probably cost about $8.6 million. Instead, the library got just $2.9 million from the SPLOST. Commissioners supplemented that with a $2 million allocation from revenue above projections — extra cash generated by the sales tax. And to top it off, the county received $1.9 million in funding from Georgia’s Board of Regents last year. That brought the budget up to $6.8 million. The county’s also elected to fold in $1.2 million set aside for the Shaw Park community center, consolidating the space into the library itself and bringing the budget to $8 million. The final $2.5 million, if approved, would come from county reserve funds. But Commissioner Keli Gambrill took issue with using general budget dollars to pay for a SPLOST project. “How many other 2016 SPLOST projects that are not completed are going to be in the same position?” she asked. “… If we we knew (the project cost) going into this, and now we’re using general funds — I mean, when’s it going to end?” Sharon Stanley, who heads up the county’s support services agency, said the previous budgets were believed to be sufficient to cover the costs in full. But rising construction prices had brought the price tag north of the $10 million mark, with contractor Batson-Cook offering the current price tag as a guaranteed ceiling. The county saw similar cost overruns with a new police precinct it’s building in northeast Cobb, which required a roughly 10% increase in its budget earlier this year. “If we don’t get funded, or this is not approved, then we’ll be back out on the market again,” said Property Management Director Travis Stalcup. “And then … we’ll just face what we’ve been facing the whole time: escalating material prices, labor shortages.” Commissioner JoAnn Birrell, who represents the area, said she would meet with her colleagues Monday to gauge support for moving forward. “If we don’t (have support), I’m pulling it,” she said. The board will meet at 7 p.m. Tuesday at 100 Cherokee Street in Marietta.	'

In [ ]:
set_merge.iloc[6]['article_final']

In [ ]:
text_ner_all(temp)

In [ ]:
text_ner_place(temp)

In [ ]:
text_ner_place2(temp)

In [ ]:
df_temp['ner_all'] = df_temp['article_final'].astype(str).apply(text_ner_all)
df_temp['ner_place_2nd'] = df_temp['article_final'].astype(str).apply(text_ner_place)

In [ ]:
df_temp['ner_place_org'] = df_temp['article_final'].astype(str).apply(text_ner_place2)

In [ ]:
df_temp

In [ ]:
df_temp[['ner_places', 'ner_place_2nd']]

In [ ]:
set_merge['ner_all'] = set_merge['article_final'].astype(str).apply(text_ner_all)

In [ ]:
set_merge['ner_place_org'] = set_merge['article_final'].astype(str).apply(text_ner_place2)

In [ ]:
set_merge.to_excel("/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/spacy_place_org.xlsx")

In [ ]:
manual = pd.read_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/Rand 35 GA articles_places mentioned.xlsx')


In [ ]:

manual['merge'] = manual.groupby(['id'])['Location'].transform(lambda x: ', '.join(x))


In [ ]:
manual = manual.drop_duplicates(subset=['id', 'merge'])

In [ ]:
manual = manual[['id', 'merge']]

In [ ]:
set_merge = pd.merge(set_merge, manual, how='left', on='id')

In [ ]:
set_merge.to_excel("/content/drive/MyDrive/Georgia/Locations Analysis/Validation_test/spacy_manual_validation.xlsx")

In [ ]:
df_sample_temp = df.sample(100)

In [ ]:
df_sample_temp

In [ ]:
# # continued from the above

# # article 
# df_sample_20 = df[ ( df['place_ng1'].str.len() >= 1) | ( df['place_ng2'].str.len() >= 1) ].sample(20)


# # place ner
# df_sample_20 = df[ ( df['place_ner1'].str.len() >= 1) | ( df['place_ner2'].str.len() >= 1) ].sample(20)

In [ ]:
## article test

temp_uni = df_sample_temp[(df_sample_temp['place_ng1'].str.len()) >= 1]
temp_bi = df_sample_temp[(df_sample_temp['place_ng2'].str.len()) >= 1]

temp_uni = temp_uni[['id', 'source', 'place_ng1', 'article_final']]
temp_bi = temp_bi[['id', 'source', 'place_ng2', 'article_final']]

temp_uni_df = temp_uni.explode('place_ng1')
temp_bi_df = temp_bi.explode('place_ng2')

temp_uni_df = temp_uni_df.rename(columns={'place_ng1': 'place_ner'})
temp_bi_df = temp_bi_df.rename(columns={'place_ng2': 'place_ner'})

## place ner test

temp_uni2 = df_sample_temp[(df_sample_temp['place_ner1'].str.len()) >= 1]
temp_bi2 = df_sample_temp[(df_sample_temp['place_ner2'].str.len()) >= 1]

temp_uni2 = temp_uni2[['id', 'source', 'place_ner1', 'article_final']]
temp_bi2 = temp_bi2[['id', 'source', 'place_ner2', 'article_final']]

temp_uni2_df = temp_uni2.explode('place_ner1')
temp_bi2_df = temp_bi2.explode('place_ner2')

temp_uni2_df = temp_uni2_df.rename(columns={'place_ner1': 'place_ner'})
temp_bi2_df = temp_bi2_df.rename(columns={'place_ner2': 'place_ner'})


In [ ]:
temp_df_all = pd.concat([temp_uni_df, temp_bi_df])
temp_df2_all = pd.concat([temp_uni2_df, temp_bi2_df])

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize
by_sentence = temp_df_all['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


In [ ]:
merge_all_join = pd.merge(temp_df_all, by_sentence_df, left_index=True, right_index=True)
merge_all_join2 = pd.merge(temp_df2_all, by_sentence_df, left_index=True, right_index=True)


In [ ]:
merge_all_join['place_YN'] = merge_all_join.apply(lambda x: x.place_ner in x.sentence_each, axis=1)
merge_all_join2['place_YN'] = merge_all_join2.apply(lambda x: x.place_ner in x.sentence_each, axis=1)

In [ ]:
merge_all_join_in = merge_all_join[ (merge_all_join['place_YN']== True)]
merge_all_join2_in = merge_all_join2[ (merge_all_join2['place_YN']== True)]


In [ ]:
location_final['place_ner'] = location_final['ner_place_clean'].copy()

In [ ]:
location_final_short = location_final[['place_ner', 'county_level', 'formatted_address']]

In [ ]:
merge_all_join_in = pd.merge(merge_all_join_in, location_final_short, how='left', on='place_ner')
merge_all_join2_in = pd.merge(merge_all_join2_in, location_final_short, how='left', on='place_ner')

In [ ]:
merge_all_join_in = merge_all_join_in[['id', 'source','place_ner', 'sentence_each', 'county_level', 'formatted_address']]
merge_all_join2_in = merge_all_join2_in[['id', 'source','place_ner', 'sentence_each', 'county_level', 'formatted_address']]

In [ ]:
merge_all_join_in = merge_all_join_in.drop_duplicates()
merge_all_join2_in = merge_all_join2_in.drop_duplicates()

In [ ]:

merge_all_join_in.to_excel('Georgia_place_sample_test_article_identification.xlsx')
merge_all_join2_in.to_excel('Georgia_place_sample_test_spacy_ner_identification.xlsx')

In [ ]:
df.sample(100)